In [ ]:
import pandas as pd
import numpy as np
import pickle

import plotly.offline as py

from joblib import Parallel, delayed
from fbprophet import Prophet
from fbprophet.plot import *
from fbprophet.diagnostics import cross_validation

pd.set_option('display.max_columns', 100)

## Params

In [ ]:
IS_EVAL = True
DATA_PATH = '../data/'

if IS_EVAL:
    PERIOD_LABEL = 'evaluation'
else:
    PERIOD_LABEL = 'validation'

## Load needed data

In [ ]:
prophet_df = pd.read_pickle(DATA_PATH + 'refined/prophet_df_' + PERIOD_LABEL + '.pkl')
prophet_params = pd.read_csv(DATA_PATH + 'external/params_prophet_store_dpt_' + PERIOD_LABEL + '.csv')
sample_submission = pd.read_csv(DATA_PATH + 'raw/sample_submission.csv')

## Forecast

In [ ]:
def forecast_prophet(store_id, dept_id):
    
    # Reduce df & params on current 
    df = prophet_df[(prophet_df['store_id'] == store_id) & (prophet_df['dept_id'] == dept_id)].copy()
    params = prophet_params.loc[(prophet_params['store_id'] == store_id) & 
                                (prophet_params['dept_id'] == dept_id), 'params'].values[0]
    params = eval(params) # String to dict
    
    # Define model
    m = Prophet(
            yearly_seasonality=False,
            weekly_seasonality=False,
            daily_seasonality=False,
            uncertainty_samples=False,
            changepoint_range=params['changepoint_range'],
            changepoint_prior_scale=params['changepoint_prior_scale'],
            holidays_prior_scale=params['holidays_prior_scale'],
            seasonality_mode=params['seasonality_mode']
        )
        
    m.add_seasonality(
        name='yearly', 
        period=365.25,
        fourier_order=params['yearly_order'],
        prior_scale=params['yearly_prior_scale']
    )
    
    m.add_seasonality(
        name='monthly', 
        period=365.25/12,
        fourier_order=params['monthly_order'],
        prior_scale=params['monthly_prior_scale']
    )
    
    m.add_seasonality(
        name='weekly', 
        period=7, 
        fourier_order=params['weekly_order'],
        prior_scale=params['weekly_prior_scale']
    )
    
    # Add holidays/regressor
    m.add_country_holidays(country_name='US')
    
    m.add_regressor('snap', mode=params['snap_mode'])
        
    m.add_regressor('price', mode=params['price_mode'])
    
    m.add_regressor('dom', mode=params['dom_mode'])
    
    # Fit 
    m.fit(df.dropna(subset=['y'])) # drop pred period
    
    # TS validation
    initial = str(df.dropna(subset=['y'])['d'].max() - 28 - 1) + ' days' # only one cutoff to validate
    df_cv = cross_validation(m, horizon='28 days', period='28 days', initial=initial)
    
    # Round forecast
    df_cv['yhat'] = df_cv['yhat'].round().astype(int)
    df_cv.loc[df_cv['yhat'] < 0, 'yhat'] = 0
    
    # Calculate Error
    score = np.mean(np.square(df_cv['y'] - df_cv['yhat']))
    scale = np.mean(np.square(np.diff(df['y'].values[:-28]).astype(np.float32)))
    rmsse = np.sqrt(score / scale)
       
    print(rmsse)

    # Predict
    future = m.make_future_dataframe(periods=28, freq='D', include_history=True)
    
    future['snap'] = df['snap'].values
    future['price'] = df['price'].values
    future['dom'] = df['dom'].values
    
    fcst = m.predict(future)
    
    # Add ids & d to the output
    fcst['store_id'] = store_id
    fcst['dept_id'] = dept_id
    fcst['d'] = df['d'].values
    
    # Round forecast
    fcst['yhat'] = fcst['yhat'].round().astype(int)
    fcst.loc[fcst['yhat'] < 0, 'yhat'] = 0
        
    return m, fcst

In [ ]:
prophet_params.sort_values('rmsse')

In [ ]:
store_id = 'CA_3'
dept_id = 'HOBBIES_2'

In [ ]:
eval(prophet_params[(prophet_params.store_id == store_id) & (prophet_params.dept_id == dept_id)].params.values[0])

In [ ]:
m, fcst = forecast_prophet(store_id, dept_id)

In [ ]:
fig = plot_plotly(m, fcst, uncertainty=True, plot_cap=False, trend=True, changepoints=True,
                  changepoints_threshold=0.01, xlabel='date', ylabel='sales', figsize=(900, 600))
py.iplot(fig)